## Longhorn Installation und Nutzung
Dieses Notebook enthält die notwendigen Befehle zur Installation, Konfiguration und Nutzung von Longhorn als verteiltes Blockspeichersystem für Kubernetes.


### Umgebungsprüfung

In [ ]:
%%bash
curl -sSfL https://raw.githubusercontent.com/longhorn/longhorn/v1.8.0/scripts/environment_check.sh | bash

### Installation von Longhorn

Dazu greifen wir auf Scripts aus dem [Lern Cloud Projekt](https://github.com/mc-b/lerncloud/tree/main/services) zurück:
* das erste deaktiviert die [Multi Path Routing](https://de.wikipedia.org/wiki/Multi-Path_Routing) Unterstützung und aktiviert [iSCSI](https://de.wikipedia.org/wiki/ISCSI)
* das zweite Installiert longhorm mittels `helm`.

Nach Durchlaufen von storage-patch.sh sollte die Umgebungsprüfung keine Fehler mehr anzeigen.

In [ ]:
%%bash
curl -sfL https://raw.githubusercontent.com/mc-b/lerncloud/main/services/storage-patch.sh | bash -
curl -sfL https://raw.githubusercontent.com/mc-b/lerncloud/main/services/longhorn.sh | bash -

### Longhorn UI

Longhorn stellt ein UI zur Verfügung.

In [ ]:
%%bash
kubectl patch service longhorn-frontend --namespace longhorn-system -p '{"spec": {"type": "NodePort"}}'
echo "http://"$(cat ~/work/server-ip)":$(kubectl get -n longhorn-system service longhorn-frontend -o=jsonpath='{ .spec.ports[0].nodePort }')/"

Die Informationen vom UI sehen wir auch als Kubernetes Ressourcen

In [ ]:
%%bash
kubectl get storageclasses,persistentvolumes,persistentvolumeclaims

Wenn die PersistentVolumeClaims `longhorn-XXX` den Status `Bound` haben, war die Installation erfolgreich.

- - -

## Testen

<img src="https://kubernetes.io/images/docs/pod.svg" width="200">

Quelle: [Pods with multiple containers ](https://kubernetes.io/docs/concepts/workloads/pods/#how-pods-manage-multiple-containers)

---

Dazu verwenden wir den Speicher mit zwei Pods, angelehnt an das [FilePuller/Web](https://kubernetes.io/docs/concepts/workloads/pods/#how-pods-manage-multiple-containers) Beispiel von Kubernetes.


In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: v1
kind: Pod
metadata:
  labels:
    app.kubernetes.io/name: web
  name: web
spec:
  containers:
  - image: registry.gitlab.com/mc-b/misegr/httpd
    name: apache
    volumeMounts:
    - mountPath: "/usr/local/apache2/htdocs"
      name: "web-storage"    
  volumes:
    - name: web-storage
      persistentVolumeClaim:
        claimName: longhorn-rwx  
EOF

In [ ]:
%%bash
kubectl apply -f - <<EOF
    apiVersion: v1
    kind: Pod
    metadata:
      labels:
        app.kubernetes.io/name: puller
      name: puller
    spec:
      containers:
      - image: registry.gitlab.com/mc-b/misegr/debian:bookworm
        name: file-puller
        # Just spin & wait forever
        command: [ "/bin/bash", "-c", "--" ]
        args: [ "while true; do echo \"<html><body><h1>Hallo es ist $(date)</h1></body></html>\" >/usr/local/apache2/htdocs/index.html; sleep 30; done;" ]    
        # Volumes im Container
        volumeMounts:
        - mountPath: "/usr/local/apache2/htdocs"
          name: "web-storage"
      # Volumes in Host      
      volumes:
        - name: web-storage
          persistentVolumeClaim:
            claimName: longhorn-rwx             
EOF

### Funktionalität prüfen

Schauen ob die Pods laufen und eine `index.html` Datei produzieren

In [ ]:
%%bash
kubectl wait --for=condition=Ready pod/puller --timeout=300s
kubectl get pods 
kubectl exec puller -- ls -l /usr/local/apache2/htdocs
kubectl exec puller -- cat /usr/local/apache2/htdocs/index.html

In der VM, Disk Image finden und Dateien darin ausgeben:

    sudo -i
    cd /var/lib/longhorn/replicas/pvc-[ID]
    debugfs volume-head-000.img
    ls -l 
    cat ....
    CTRL+d

Nach erfolgreichem Test können die Pods wieder gelöscht werden:

In [ ]:
%%bash
kubectl delete pod puller
kubectl delete pod web

***

## Aufträge
- Richtet Longhorn als "default" Storage Class ein.
- Portiert Eure Microservices inkl. Dateiablage auf Longhorn, was sind die Vorteile?
- Überlegt Euch ein Backup Konzept mit longhorn und setzt dieses um.

**Links:**
- [Offizielle Longhorn-Dokumentation](https://longhorn.io/docs/1.8.0/deploy/install/)
- [Fehlerbehebung in Longhorn](https://longhorn.io/docs/1.8.0/troubleshooting/)

- - -
## Aufräumen

* Longhorn entfernen.
* Eine "Default" Storage Class setzen


In [ ]:
%%bash
kubectl delete PersistentVolumeClaim longhorn-rwo || true
kubectl delete PersistentVolumeClaim longhorn-rwx || true
kubectl -n longhorn-system patch settings.longhorn.io deleting-confirmation-flag --type=merge -p '{"value":"true"}'
helm uninstall longhorn --namespace longhorn-

# default StorageClass
# kubectl patch storageclass microk8s-hostpath -p '{"metadata": {"annotations":{"storageclass.kubernetes.io/is-default-class":"true"}}}'
